# Text Classification Model for Large Movie Review Using TensorFlow Take 2
### David Lowe
### March 12, 2021

Template Credit: Adapted from a template made available by Dr. Jason Brownlee of Machine Learning Mastery. [https://machinelearningmastery.com/]

SUMMARY: This project aims to construct a text classification model using a neural network and document the end-to-end steps using a template. The Large Movie Review dataset is a binary classification situation where we attempt to predict one of the two possible outcomes.

INTRODUCTION: The Large Movie Review Dataset is a collection of movie reviews used in the research paper "Learning Word Vectors for Sentiment Analysis" by Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts, The 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011). The dataset comprises 25,000 highly polar movie reviews for training and 25,000 for testing.

From iteration Take1, we constructed a bag-of-words model and analyzed it with a simple TensorFlow deep learning network. Due to the system's memory limitation, we had to break up the script processing into two parts. Part A tested the model with the training dataset using a five-fold validation. Part B trained the final model with the entire training dataset and made predictions on a previously unseen test dataset.

This Take2 iteration will construct a word-embedding model and analyze it with a simple TensorFlow deep learning network.

ANALYSIS: From iteration Take1, the bag-of-words model's performance achieved an average accuracy score of 87.18% after 20 epochs with five iterations of cross-validation. Furthermore, the final model processed the test dataset with an accuracy measurement of 85.24%.

In this Take2 iteration, the word-embedding model's performance achieved an average accuracy score of 88.21% after 10 epochs with five iterations of cross-validation. Furthermore, the final model processed the test dataset with an accuracy measurement of 85.84%.

CONCLUSION: In this modeling iteration, the word-embedding TensorFlow model appeared to be suitable for modeling this dataset. We should consider experimenting with TensorFlow for further modeling.

Dataset Used: Large Movie Review Dataset

Dataset ML Model: Binary class text classification with text-oriented features

Dataset Reference: https://ai.stanford.edu/~amaas/papers/wvSent_acl2011.bib

One potential source of performance benchmarks: https://ai.stanford.edu/~amaas/data/sentiment/ and https://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf

A deep-learning text classification project generally can be broken down into five major tasks:

1. Prepare Environment
2. Load and Prepare Text Data
3. Define and Train Models
4. Evaluate and Optimize Models
5. Finalize Model and Make Predictions

# Task 1 - Prepare Environment

In [1]:
# # Install the packages to support accessing environment variable and SQL databases
# !pip install python-dotenv PyMySQL boto3

In [2]:
# Retrieve GPU configuration information from Colab
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

Thu Mar  4 22:36:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Retrieve memory configuration information from Colab
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
    print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
    print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
    print('re-execute this cell.')
else:
    print('You are using a high-RAM runtime!')

Your runtime has 13.7 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime → "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


In [4]:
# Retrieve CPU information from the system
ncpu = !nproc
print("The number of available CPUs is:", ncpu[0])

The number of available CPUs is: 2


## 1.a) Load libraries and modules

In [5]:
# Set the random seed number for reproducible results
seedNum = 888

In [6]:
# Load libraries and packages
import random
random.seed(seedNum)
import numpy as np
np.random.seed(seedNum)
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import sys
# import boto3
import shutil
import string
import nltk
from nltk.corpus import stopwords
from collections import Counter
from datetime import datetime
# from dotenv import load_dotenv
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import tensorflow as tf
tf.random.set_seed(seedNum)
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [7]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

## 1.b) Set up the controlling parameters and functions

In [8]:
# Begin the timer for the script processing
startTimeScript = datetime.now()

# Set up the number of CPU cores available for multi-thread processing
n_jobs = 1

# Set up the flag to stop sending progress emails (setting to True will send status emails!)
notifyStatus = False

# Set Pandas options
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 140)

# Set the percentage sizes for splitting the dataset
test_set_size = 0.2
val_set_size = 0.25

# Set the number of folds for cross validation
n_folds = 5
n_iterations = 1

# Set various default modeling parameters
default_loss = 'binary_crossentropy'
default_metrics = ['accuracy']
default_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
default_kernel_init = tf.keras.initializers.GlorotUniform(seed=seedNum)
default_epochs = 10
default_batch_size = 64
default_vector_space = 100
default_filters = 32
default_kernel_size = 8
default_pool_size = 2

# Define the labels to use for graphing the data
train_metric = "accuracy"
validation_metric = "val_accuracy"
train_loss = "loss"
validation_loss = "val_loss"

# Check the number of GPUs accessible through TensorFlow
print('Num GPUs Available:', len(tf.config.list_physical_devices('GPU')))

# Print out the TensorFlow version for confirmation
print('TensorFlow version:', tf.__version__)

Num GPUs Available: 1
TensorFlow version: 2.4.1


In [9]:
# Set up the parent directory location for loading the dotenv files
# from google.colab import drive
# drive.mount('/content/gdrive')
# gdrivePrefix = '/content/gdrive/My Drive/Colab_Downloads/'
# env_path = '/content/gdrive/My Drive/Colab Notebooks/'
# dotenv_path = env_path + "python_script.env"
# load_dotenv(dotenv_path=dotenv_path)

# Set up the dotenv file for retrieving environment variables
# env_path = "/Users/david/PycharmProjects/"
# dotenv_path = env_path + "python_script.env"
# load_dotenv(dotenv_path=dotenv_path)

In [10]:
# Set up the email notification function
def status_notify(msg_text):
    access_key = os.environ.get('SNS_ACCESS_KEY')
    secret_key = os.environ.get('SNS_SECRET_KEY')
    aws_region = os.environ.get('SNS_AWS_REGION')
    topic_arn = os.environ.get('SNS_TOPIC_ARN')
    if (access_key is None) or (secret_key is None) or (aws_region is None):
        sys.exit("Incomplete notification setup info. Script Processing Aborted!!!")
    sns = boto3.client('sns', aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=aws_region)
    response = sns.publish(TopicArn=topic_arn, Message=msg_text)
    if response['ResponseMetadata']['HTTPStatusCode'] != 200 :
        print('Status notification not OK with HTTP status code:', response['ResponseMetadata']['HTTPStatusCode'])

In [11]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 1 - Prepare Environment has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [12]:
# Reset the random number generators
def reset_random(x):
    random.seed(x)
    np.random.seed(x)
    tf.random.set_seed(x)

In [13]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 1 - Prepare Environment completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 2 - Load and Prepare Text Data

In [14]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 2 - Load and Prepare Text Data has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

## 2.a) Download Text Data Archive

In [15]:
# Clean up the old files and download directories before receiving new ones
!rm -rf staging/
!rm aclImdb_v1.tar.gz
!rm vocabulary.txt

In [16]:
!wget https://dainesanalytics.com/datasets/acl2011-large-movie-review/aclImdb_v1.tar.gz

--2021-03-04 22:36:13--  https://dainesanalytics.com/datasets/acl2011-large-movie-review/aclImdb_v1.tar.gz
Resolving dainesanalytics.com (dainesanalytics.com)... 13.226.50.123, 13.226.50.115, 13.226.50.69, ...
Connecting to dainesanalytics.com (dainesanalytics.com)|13.226.50.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  19.2MB/s    in 4.1s    

2021-03-04 22:36:17 (19.7 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



## 2.b) Splitting Data for Training and Validation

In [17]:
staging_dir = 'staging/'
!mkdir staging/

In [18]:
local_archive = 'aclImdb_v1.tar.gz'
shutil.unpack_archive(local_archive, staging_dir)

In [19]:
training_dir = 'staging/aclImdb/train/'
testing_dir = 'staging/aclImdb/test/'
classA_name = 'pos'
classB_name = 'neg'

In [20]:
# Brief listing of training text files for both classes before splitting
training_classA_dir = os.path.join(training_dir, classA_name)
training_classA_files = os.listdir(training_classA_dir)
print('Number of training images for', classA_name, ':', len(training_classA_files))
print('Training samples for', classA_name, ':', training_classA_files[:10])

training_classB_dir = os.path.join(training_dir, classB_name)
training_classB_files = os.listdir(training_classB_dir)
print('Number of training images for', classB_name, ':', len(training_classB_files))
print('Training samples for', classB_name, ':', training_classB_files[:10])

Number of training images for pos : 12500
Training samples for pos : ['8450_7.txt', '2527_9.txt', '1061_10.txt', '4335_9.txt', '8689_7.txt', '1033_10.txt', '8834_9.txt', '8108_10.txt', '3671_7.txt', '141_9.txt']
Number of training images for neg : 12500
Training samples for neg : ['12037_1.txt', '2627_1.txt', '11764_3.txt', '9476_4.txt', '5526_3.txt', '5015_1.txt', '4672_3.txt', '2522_4.txt', '12303_1.txt', '10561_1.txt']


In [21]:
# Brief listing of testing text files for both classes after splitting
testing_classA_dir = os.path.join(testing_dir, classA_name)
testing_classA_files = os.listdir(testing_classA_dir)
print('Number of testing files for', classA_name, ':', len(testing_classA_files))
print('Test samples for', classA_name, ':', testing_classA_files[:10])

testing_classB_dir = os.path.join(testing_dir, classB_name)
testing_classB_files = os.listdir(testing_classB_dir)
print('Number of testing files for', classB_name, ':', len(testing_classB_files))
print('Test samples for', classB_name, ':', testing_classB_files[:10])

Number of testing files for pos : 12500
Test samples for pos : ['8450_7.txt', '11466_8.txt', '2344_10.txt', '4269_7.txt', '2527_9.txt', '6721_8.txt', '1429_7.txt', '6138_10.txt', '4335_9.txt', '4915_10.txt']
Number of testing files for neg : 12500
Test samples for neg : ['1239_4.txt', '10653_1.txt', '6102_1.txt', '12370_1.txt', '6773_4.txt', '10541_2.txt', '6242_3.txt', '4386_1.txt', '4203_3.txt', '5015_1.txt']


## 2.c) Load Document and Build Vocabulary

In [22]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [23]:
# turn a doc into clean tokens
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

In [24]:
# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
    # load doc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # update counts
    vocab.update(tokens)

In [25]:
# load all docs in a directory
def build_vocab_from_docs(directory, vocab):
    # walk through all files in the folder
    i = 0
    print('Processing the text files and showing the first 10...')
    for filename in os.listdir(directory):
        # skip files that do not have the right extension
        if not filename.endswith(".txt"):
            continue
        # create the full path of the file to open
        path = directory + '/' + filename
        # add doc to vocab
        add_doc_to_vocab(path, vocab)
        i = i + 1
        if i < 10: print('Loaded %s' % path)
    print('Total number of text files loaded:', i, '\n')

In [26]:
# define vocab
vocab = Counter()
# add all docs to vocab
build_vocab_from_docs(training_classA_dir, vocab)
build_vocab_from_docs(training_classB_dir, vocab)
# print the size of the vocab
print('The total number of words in the vocabulary:', len(vocab))
# print the top words in the vocab
top_words = 50
print('The top', top_words, 'words in the vocabulary:\n', vocab.most_common(top_words))

Processing the text files and showing the first 10...
Loaded staging/aclImdb/train/pos/8450_7.txt
Loaded staging/aclImdb/train/pos/2527_9.txt
Loaded staging/aclImdb/train/pos/1061_10.txt
Loaded staging/aclImdb/train/pos/4335_9.txt
Loaded staging/aclImdb/train/pos/8689_7.txt
Loaded staging/aclImdb/train/pos/1033_10.txt
Loaded staging/aclImdb/train/pos/8834_9.txt
Loaded staging/aclImdb/train/pos/8108_10.txt
Loaded staging/aclImdb/train/pos/3671_7.txt
Total number of text files loaded: 12500 

Processing the text files and showing the first 10...
Loaded staging/aclImdb/train/neg/12037_1.txt
Loaded staging/aclImdb/train/neg/2627_1.txt
Loaded staging/aclImdb/train/neg/11764_3.txt
Loaded staging/aclImdb/train/neg/9476_4.txt
Loaded staging/aclImdb/train/neg/5526_3.txt
Loaded staging/aclImdb/train/neg/5015_1.txt
Loaded staging/aclImdb/train/neg/4672_3.txt
Loaded staging/aclImdb/train/neg/2522_4.txt
Loaded staging/aclImdb/train/neg/12303_1.txt
Total number of text files loaded: 12500 

The tota

In [27]:
# keep tokens with a min occurrence
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print('The number of words with the minimum appearance:', len(tokens))

The number of words with the minimum appearance: 65663


In [28]:
# save list to file
def save_list(lines, filename):
    # convert lines to a single blob of text
    data = '\n'.join(lines)
    # open file
    file = open(filename, 'w')
    # write text
    file.write(data)
    # close file
    file.close()

# save tokens to a vocabulary file
save_list(tokens, 'vocabulary.txt')

## 2.d) Create Tokenizer and Encode the Input Text

In [29]:
# load doc, clean and return line of tokens
def doc_to_line(filename, vocab):
    # load the doc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # filter by vocab
    tokens = [w for w in tokens if w in vocab]
    return ' '.join(tokens)

In [30]:
# load all docs in a directory
def process_docs_to_lines(directory, vocab):
    lines = list()
    # walk through all files in the folder
    for filename in os.listdir(directory):
        # create the full path of the file to open
        path = directory + '/' + filename
        # load and clean the doc
        line = doc_to_line(path, vocab)
        # add to list
        lines.append(line)
    return lines

In [31]:
# load the vocabulary
vocab_filename = 'vocabulary.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
print('Number of tokens in the vocabulary:', len(vocab))

Number of tokens in the vocabulary: 65663


In [32]:
# prepare bag of words encoding of docs
def encode_training_data(train_docs, maxlen):
    # create the tokenizer
    tokenizer = Tokenizer()
    # fit the tokenizer on the documents
    tokenizer.fit_on_texts(train_docs)
    # encode training data set
    encoded_docs = tokenizer.texts_to_sequences(train_docs)
    # pad sequences
    train_encoded = pad_sequences(encoded_docs, maxlen=maxlen, padding='post')
    # define vocabulary size (largest integer value)
    vocab_size = len(tokenizer.word_index) + 1
    return train_encoded, vocab_size

In [33]:
# Load all training cases
positive_train_cases = process_docs_to_lines(training_classA_dir, vocab)
print('The number of positive reviews processed:', len(positive_train_cases))
negative_train_cases = process_docs_to_lines(training_classB_dir, vocab)
print('The number of negative reviews processed:', len(negative_train_cases))
training_docs =  negative_train_cases + positive_train_cases
y_train = np.array([0 for _ in range(len(negative_train_cases))] + [1 for _ in range(len(positive_train_cases))])
print('The shape of the encoded training classes:', y_train.shape)

The number of positive reviews processed: 12500
The number of negative reviews processed: 12500
The shape of the encoded training classes: (25000,)


In [34]:
# Get maximum doc length for padding sequences
max_length = max([len(s.split()) for s in training_docs])
print('The maximum document length:', max_length)

The maximum document length: 1447


In [35]:
# encode training datasets
X_train, vocabulary_size = encode_training_data(training_docs, max_length)
print('The vocabulary size:', vocabulary_size)
print('The shape of the encoded training dataset:', X_train.shape)

The vocabulary size: 54889
The shape of the encoded training dataset: (25000, 1447)


In [36]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 2 - Load and Prepare Text Data completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 3 - Define and Train Models

In [37]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 3 - Define and Train Models has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [38]:
# Define the default numbers of input/output for modeling
num_outputs = 1

In [39]:
# Define the baseline model for benchmarking
def create_nn_model(n_inputs=max_length, n_outputs=num_outputs, n_vocab=vocabulary_size, n_vectors=default_vector_space, n_pools=default_pool_size, 
                    conv1_filters=default_filters, conv1_kernels=default_kernel_size, dense_nodes=256, opt_param=default_optimizer, init_param=default_kernel_init):
    nn_model = keras.Sequential([
        layers.Embedding(n_vocab, n_vectors, input_length=n_inputs),
        layers.Conv1D(filters=conv1_filters, kernel_size=conv1_kernels, activation='relu'),
        layers.MaxPooling1D(pool_size=n_pools),
        layers.Flatten(),
        layers.Dense(dense_nodes, activation='relu', kernel_initializer=init_param),
        layers.Dense(n_outputs, activation='sigmoid', kernel_initializer=init_param)
    ])
    nn_model.compile(loss=default_loss, optimizer=opt_param, metrics=default_metrics)
    return nn_model

In [40]:
# Initialize the default model and get a baseline result
startTimeModule = datetime.now()
results = list()
iteration = 0
cv = RepeatedKFold(n_splits=n_folds, n_repeats=n_iterations, random_state=seedNum)
for train_ix, val_ix in cv.split(X_train):
    feature_train, feature_validation = X_train[train_ix], X_train[val_ix]
    target_train, target_validation = y_train[train_ix], y_train[val_ix]
    reset_random(seedNum)
    baseline_model = create_nn_model()
    baseline_model.fit(feature_train, target_train, epochs=default_epochs, batch_size=default_batch_size, verbose=1)
    model_metric = baseline_model.evaluate(feature_validation, target_validation, verbose=0)[1]
    iteration = iteration + 1
    print('Accuracy measurement from iteration %d >>> %.2f%%' % (iteration, model_metric*100))
    results.append(model_metric)
validation_score = np.mean(results)
validation_variance = np.std(results)
print('Average model accuracy from all validation iterations: %.2f%% (%.2f%%)' % (validation_score*100, validation_variance*100))
print('Total time for model fitting and cross validating:', (datetime.now() - startTimeModule))

Epoch 1/10
313/313 [==============================] - 25s 75ms/step - loss: 0.5471 - accuracy: 0.6682
Epoch 2/10
313/313 [==============================] - 23s 74ms/step - loss: 0.1227 - accuracy: 0.9568
Epoch 3/10
313/313 [==============================] - 23s 74ms/step - loss: 0.0245 - accuracy: 0.9941
Epoch 4/10
313/313 [==============================] - 24s 75ms/step - loss: 0.0034 - accuracy: 0.9995
Epoch 5/10
313/313 [==============================] - 24s 76ms/step - loss: 6.4197e-04 - accuracy: 0.9999
Epoch 6/10
313/313 [==============================] - 24s 76ms/step - loss: 1.4765e-04 - accuracy: 1.0000
Epoch 7/10
313/313 [==============================] - 24s 77ms/step - loss: 6.6335e-05 - accuracy: 1.0000
Epoch 8/10
313/313 [==============================] - 24s 77ms/step - loss: 4.1698e-05 - accuracy: 1.0000
Epoch 9/10
313/313 [==============================] - 24s 77ms/step - loss: 2.4217e-05 - accuracy: 1.0000
Epoch 10/10
313/313 [==============================] - 24s 76m

In [41]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 3 - Define and Train Models completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 4 - Evaluate and Optimize Models

In [42]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 4 - Evaluate and Optimize Models has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [43]:
# Not applicable for this iteration of modeling

In [44]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 4 - Evaluate and Optimize Models completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 5 - Finalize Model and Make Predictions

In [45]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 5 - Finalize Model and Make Predictions has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

## 5.a) Train the Final Model

In [46]:
# Create the final model for evaluating the test dataset
reset_random(seedNum)
final_model = create_nn_model()
final_model.fit(X_train, y_train, epochs=default_epochs, batch_size=default_batch_size, verbose=1)

Epoch 1/10
391/391 [==============================] - 31s 78ms/step - loss: 0.6192 - accuracy: 0.7177
Epoch 2/10
391/391 [==============================] - 30s 77ms/step - loss: 0.1022 - accuracy: 0.9666
Epoch 3/10
391/391 [==============================] - 30s 77ms/step - loss: 0.0225 - accuracy: 0.9951
Epoch 4/10
391/391 [==============================] - 30s 77ms/step - loss: 0.0027 - accuracy: 0.9993
Epoch 5/10
391/391 [==============================] - 30s 78ms/step - loss: 7.2270e-04 - accuracy: 0.9997
Epoch 6/10
391/391 [==============================] - 30s 77ms/step - loss: 2.2573e-04 - accuracy: 1.0000
Epoch 7/10
391/391 [==============================] - 30s 78ms/step - loss: 3.1779e-04 - accuracy: 0.9999
Epoch 8/10
391/391 [==============================] - 30s 78ms/step - loss: 7.8314e-05 - accuracy: 1.0000
Epoch 9/10
391/391 [==============================] - 31s 78ms/step - loss: 6.8312e-05 - accuracy: 1.0000
Epoch 10/10
391/391 [==============================] - 30s 78m

In [47]:
# Summarize the final model
final_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1447, 100)         5488900   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 1440, 32)          25632     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 720, 32)           0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 23040)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               5898496   
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 257       
Total params: 11,413,285
Trainable params: 11,413,285
Non-trainable params: 0
__________________________________________

## 5.b) Make Predictions on Test Dataset

In [48]:
# load all test cases
positive_test_cases = process_docs_to_lines(testing_classA_dir, vocab)
print('The number of positive reviews processed:', len(positive_test_cases))
negative_test_cases = process_docs_to_lines(testing_classB_dir, vocab)
print('The number of negative reviews processed:', len(negative_test_cases))
testing_docs = negative_test_cases + positive_test_cases
y_test = np.array([0 for _ in range(len(negative_test_cases))] + [1 for _ in range(len(positive_test_cases))])
print('The shape of the encoded test classes:', y_test.shape)

The number of positive reviews processed: 12500
The number of negative reviews processed: 12500
The shape of the encoded test classes: (25000,)


In [49]:
# prepare bag of words encoding of docs
def encode_test_data(train_docs, val_docs, maxlen):
    # create the tokenizer
    tokenizer = Tokenizer()
    # fit the tokenizer on the documents
    tokenizer.fit_on_texts(train_docs)
    # encode validation data set
    encoded_docs = tokenizer.texts_to_sequences(val_docs)
    # pad sequences
    val_encoded = pad_sequences(encoded_docs, maxlen=maxlen, padding='post')
    # define vocabulary size (largest integer value)
    vocab_size = len(tokenizer.word_index) + 1
    return val_encoded, vocab_size

In [50]:
# encode test datasets
X_test, vocabulary_size = encode_test_data(training_docs, testing_docs, max_length)
print('The shape of the encoded test dataset:', X_test.shape)

The shape of the encoded test dataset: (25000, 1447)


In [51]:
# test_predictions = final_model.predict(X_test, batch_size=default_batch, verbose=1)
test_predictions = (final_model.predict(X_test) > 0.5).astype("int32").ravel()
print('Accuracy Score:', accuracy_score(y_test, test_predictions))
print(confusion_matrix(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

Accuracy Score: 0.85844
[[10870  1630]
 [ 1909 10591]]
              precision    recall  f1-score   support

           0       0.85      0.87      0.86     12500
           1       0.87      0.85      0.86     12500

    accuracy                           0.86     25000
   macro avg       0.86      0.86      0.86     25000
weighted avg       0.86      0.86      0.86     25000



In [52]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 5 - Finalize Model and Make Predictions completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [53]:
print ('Total time for the script:',(datetime.now() - startTimeScript))

Total time for the script: 0:26:06.959112
